In [37]:
import tensorflow as tf

import numpy as np
import os
import time

In [38]:
path_to_file = '/content/processed texts.csv'
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# The unique characters in the file
vocab = sorted(set(text))

In [39]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [40]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [41]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [42]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [43]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [44]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [45]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [46]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [47]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           44288     
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_1 (Dense)              (64, None, 173)           177325    
Total params: 4,159,917
Trainable params: 4,159,917
Non-trainable params: 0
_________________________________________________________________


In [48]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [49]:
model.compile(optimizer='adam', loss=loss)

In [50]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [51]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
4386/4386 [==============================] - 607s 138ms/step - loss: 1.3850
Epoch 2/10
4386/4386 [==============================] - 607s 138ms/step - loss: 1.1869
Epoch 3/10
4386/4386 [==============================] - 607s 138ms/step - loss: 1.1649
Epoch 4/10
4386/4386 [==============================] - 607s 138ms/step - loss: 1.1576
Epoch 5/10
4386/4386 [==============================] - 607s 139ms/step - loss: 1.1565
Epoch 6/10
4386/4386 [==============================] - 605s 138ms/step - loss: 1.1602
Epoch 7/10
4386/4386 [==============================] - 603s 137ms/step - loss: 1.1708
Epoch 8/10
4386/4386 [==============================] - 602s 137ms/step - loss: 1.1890
Epoch 9/10
4386/4386 [==============================] - 601s 137ms/step - loss: 1.5330
Epoch 10/10
4386/4386 [==============================] - 600s 137ms/step - loss: 1.8683


In [69]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [71]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights('./training_checkpoints/ckpt_5')

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

In [73]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [74]:
print(generate_text(model, start_string=u"Schools kill creativity: \n"))

Schools kill creativity: 
, no. Now, Yegule-50 including the Kouri had made an electric employee knocker, and we realized Arctic Ocean, Tann Hadyad joy looks up. They boarding same fonce or that if you discover the stories: You'll see an understanding of streets. While you're all truth scored our police. And when their jails off. Every now freet out of people with an entry supporters this work found less lucky. All that will make jobs, it can parkingly see is complicated, but why just notice his community reclaim. And just last five: Vat do, a student didn't actually get rid of this. The Phesh Alloxurbaumad main theoretic medilar this amazing region bodies, laboration right?(Applause)Make it from that internet. It's everywhere. It just gives a hurographs on the body. It's now plumed versions and gains for another. But across the screen, te, million people in the debecant of every examilla — a nurrazing bottom.MW: Yes, Carolina. He said, "It is now where every best fractures to end, but